In [10]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime

datapath='Processed_Data/city_compare.h5'
db=pd.HDFStore(datapath)
db.keys()

airports=db['airports']
amap=db['amap']
hdl=db['hdl']
heytea=db['heytea']
hotel=db['hotel']
subway=db['subway']
movie=db['movie']

db.close()

In [20]:
city_popu=pd.read_hdf('./Processed_Data/citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])
city_popu

,城市,GDP,人均可支配收入,常住人口
0,合肥,10046,48283,512
1,阜阳,2805,34562,213
2,安庆,2468,35947,80
3,六安,1670,33647,197
4,滁州,3032,36051,78
...,...,...,...,...
292,丽水,1540,48532,56
293,嘉兴,5510,64124,152
294,舟山,1512,63702,88
295,绍兴,6001,66694,300


In [21]:
df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
df_covid=df_covid[['city','peak','progress_2']]
df_covid.columns=['城市','峰值日','covid_progess']
df_covid['covid_progess']=pd.to_numeric(df_covid['covid_progess'])/100
df_covid.head(2)

,城市,峰值日,covid_progess
0,日喀则,2022/12/24,0.0
1,镇江,2022/12/21,0.0


In [32]:
all=city_popu.merge(df_covid,how='outer',on='城市').merge(amap.drop(columns='date'),how='outer',on='城市').merge(airports.drop(columns='date'),how='outer',on='城市').merge(subway.drop(columns='date'),how='outer',on='城市').merge(heytea.drop(columns='date'),how='outer',on='城市').merge(movie,how='outer',on='城市').merge(hdl.drop(columns='date'),how='outer',on='城市').merge(hotel.drop(columns='date'),how='outer',on='城市')


all['N']=all.count(axis=1)
all=all[all['covid_progess']>=0]

all=all.sort_values(by='N',ascending=False).head(1000).sort_values(by='常住人口',ascending=False).head(1000).drop(columns='N')
all.drop(columns='峰值日').to_clipboard(index=False)


In [33]:
# d1=pd.read_clipboard()
# d1.to_hdf('Processed_Data/tablehead.h5','all')
d1.merge(all,on='城市',how='left').to_clipboard()

In [34]:
%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi
all[all['城市'].isin(dafeng_city)].to_clipboard()
all[all['城市'].isin(['成都'])]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,城市,GDP,人均可支配收入,常住人口,峰值日,covid_progess,amap,airports,subway,heytea,movie,hdl,hotel,N
248,成都,17717.0,48593.0,1543.0,2022/12/15,0.0,1.137205,1.012205,1.182154,1.172542,0.68,0.633111,1.076193,13
